# Partialing Prompt Templates

Cassandra-powered prompt templates support partialing. Building on the previous examples:

In [1]:
from langchain.prompts.database import CassandraReaderPromptTemplate

In [2]:
from cqlsession import getCQLSession, getCQLKeyspace
cqlMode = 'astra_db' # 'astra_db'/'local'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

In [3]:
ctemplate0 = """Please answer a question from a user.
Keep in mind that the user's age is {user_age} and they live in a city with
nickname {city_nickname}.

USER'S QUESTION: {user_question}
YOUR ANSWER:
"""

cassPrompt = CassandraReaderPromptTemplate(
    session=session,
    keyspace=keyspace,
    template=ctemplate0,
    input_variables=['user_question'],
    field_mapper={
        'user_age': ('people', 'age'),
        'city_nickname': ('nickname_by_city', 'nickname'),
    },
)

VALIDATING IN CRPT
VALIDATING IN CPT


## Creating a Partial Prompt

Let us "partial" the prompt above, specifying the DB-lookup inputs and leaving only the user question unspecified:

In [4]:
cassPartialPrompt = cassPrompt.partial(city='lisbon', name='Pedro')

VALIDATING IN CRPT
VALIDATING IN CPT


When rendering the template, the full lookup takes place at once:

In [5]:
print(cassPartialPrompt.format(user_question='Em verdade, o que quiseres?'))

CALLING CONVERTOR {'city': 'lisbon', 'name': 'Pedro'}
Please answer a question from a user.
Keep in mind that the user's age is 1 and they live in a city with
nickname ACidade.

USER'S QUESTION: Em verdade, o que quiseres?
YOUR ANSWER:



## Partialing and Chat Prompt Templates

In most cases you may want to partial with respect to the database lookup key(s) even within a chat prompt template:

In [10]:
systemTemplate = """
You are a chat assistant, helping a user of age {user_age} from a city
they refer to as {city_nickname}.
"""

cassSystemPrompt = CassandraReaderPromptTemplate(
    session=session,
    keyspace=keyspace,
    template=systemTemplate,
    input_variables=[],
    field_mapper={
        'user_age': ('people', 'age'),
        'city_nickname': ('nickname_by_city', 'nickname'),
    },
)

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
systemMessagePrompt = SystemMessagePromptTemplate(prompt=cassSystemPrompt)

humanTemplate = "{text}"
humanMessagePrompt = HumanMessagePromptTemplate.from_template(humanTemplate)

cassChatPrompt = ChatPromptTemplate.from_messages(
    [systemMessagePrompt, humanMessagePrompt]
)

VALIDATING IN CRPT
VALIDATING IN CPT


and try to "partial" it by specifying the DB-lookup parameters:

In [11]:
cassChatPartialPrompt = cassChatPrompt.partial(
    city='turin',
    name='beppe'
)

In [12]:
cassChatPartialPrompt.format(text="Hahaha!")

CALLING CONVERTOR {'city': 'turin', 'name': 'beppe'}


'System: \nYou are a chat assistant, helping a user of age 2 from a city\nthey refer to as CereaNeh.\n\nHuman: Hahaha!'